In [2]:
import numpy as np

Updated Drag Polar

In [3]:
S_ref = 312 #ft^3
k = 1.4 
rho = 1.969 * 10**-3
T = 490.1
R = 53.35
v = 123.171
visc = 3.575 * 10**-7

a = np.sqrt(k * R * T)
M = v / a 


In [4]:
def getFF(xc_max, f, lam, FF_type):
    #FF_type = 1 for wings, tails, struts and pylons
    #FF_type = 2 for fuselage and canopy, f < 6
    #FF_type = 3 for fuselage and canopy, f > 6
    #FF_type = 4 for nacelles
    if(FF_type == 1):
        FF  = (1 + 0.6 / xc_max * f + 100 * f**4) * (1.34 * M**0.18 * np.cos(lam)**0.28)
    elif(FF_type == 2):
        FF = 0.9  + 5 / f** 1.5 + f/400
    elif(FF_type == 3):
        FF = 1 + 60/ f**3 + f /400
    else:
        FF = 1 + 0.35 / f
    return FF

def getCf(lc, comp_type):
    #comp_type = 1 for fuselage
    #comp_type = 2 for everything else
    if comp_type == 1:
        lam_rat = 0.25
    else: 
        lam_rat = 0.5

    Re = rho * v * lc / visc

    Cf_lam = 1.328 / np.sqrt(Re)
    Cf_turb = 0.455 / (np.log10(Re)**2.58 * (1 + 0.144 * M ** 2)** 0.65)

    return (Cf_lam * lam_rat + Cf_turb * (1 - lam_rat))

def getCompCD0(Cf, FF, Q, S_wet):
    return (Cf * FF * Q * S_wet / S_ref)

In [5]:

# Component A: Fuselage
Swet_A = 376.2
Q_A = 1.0
l_A = 30.0
f_A = 4.832 #from VSP
maxcamb_loc_A = 0
lambda_A = 0
FF_A = getFF(0, f_A , 0, 2)
Cf_A = getCf(l_A, 1)

Cd0_A = getCompCD0(Cf_A, FF_A, Q_A, Swet_A)



#Component B: Main Wing (NACA 23018)
Swet_B = 538.5
Q_B = 1.0
l_B = 6.22
f_B = 0.18
maxcamb_loc_B = 0.15
lambda_B = np.arctan(2.1 / 13.0)
FF_B = getFF(maxcamb_loc_B, f_B , lambda_B, 1)
Cf_b = getCf(l_B, 2)

Cd0_B = getCompCD0(Cf_b, FF_B, Q_B, Swet_B)


#Component C: Canard (NACA 0012)
Swet_C = 83.49
Q_C = 1.0
l_C = 2.67
f_C = 0.12  
maxcamb_loc_C = 0.3
lambda_C = 0
FF_C = getFF(maxcamb_loc_C, f_C , lambda_C, 1)
Cf_C = getCf(l_C, 2)

Cd0_C = getCompCD0(Cf_C, FF_C, Q_C, Swet_C)


#Component D: Canopy
Swet_D = 57.03
Q_D = 1.0
l_D = 12.0
f_D = 2.635 #from VSP
maxcamb_loc_D = 0
lambda_D = 0
FF_D = getFF(0, f_D , 0, 2)
Cf_D = getCf(l_D, 1)

Cd0_D = getCompCD0(Cf_A, FF_A, Q_A, Swet_A)



#Component E: Booms
Swet_E = 79.91
Q_E = 1.0
l_E = 18.0
d_E = 1.0
f_E = l_E / d_E
maxcamb_loc_E = 0
lambda_E = 0
FF_E = getFF(0, f_E ,0, 3)
Cf_E = getCf(l_E, 2)

Cd0_E = getCompCD0(Cf_E, FF_E, Q_E, Swet_E)

#Component F: Main Winglet (NACA 0012)
Swet_F = 46.39
Q_F = 1.0
l_F = 3.16
f_F = 0.12
maxcamb_loc_F = 0.2
lambda_F = np.arctan(3.35/1.63)
FF_F = getFF(maxcamb_loc_F, f_F , lambda_F, 1)
Cf_F = getCf(l_F, 2)

Cd0_F = getCompCD0(Cf_F, FF_F, Q_F, Swet_F)


#Component G: Canard Winglet (NACA 0012)
Swet_G = 8.17
Q_G = 1.0
l_G = 2.07
f_G = 0.12
maxcamb_loc_G = 0.2
lambda_G = 0
FF_G = getFF(maxcamb_loc_G, f_G , lambda_G, 1)
Cf_G  = getCf(l_G, 2)

Cd0_G = getCompCD0(Cf_G, FF_G, Q_G, Swet_G)


#Component H: Electric Nacelles
Swet_H = [10.04, 10.14, 11.17, 12.72, 10.82]
Q_H = 1.0
l_H = 5.0
d_H = 2/3
f_H = l_H / d_H
maxcamb_loc_G = 0
lambda_H = 0
FF_H = getFF(0, f_H , 0, 4)
Cf_H  = getCf(l_H, 2)

Cd0_H = np.zeros(5)

for i in range(5):
    Cd0_H[i] = getCompCD0(Cf_H, FF_H, Q_H, Swet_H[i])


#Component I: Horizontal Stabilizer
Swet_I = 81.09
Q_I = 1.07
l_I = 2.5
f_I = 0.1  
maxcamb_loc_I = 0.2
lambda_I = 0
FF_I = getFF(maxcamb_loc_I, f_I , lambda_I, 1)
Cf_I = getCf(l_I, 2)

Cd0_I = getCompCD0(Cf_I, FF_I, Q_I, Swet_I)

#Component J: Vertical Stabilizers
Sref_low = (5 + 28/9) / 2 * 8/3
Sref_high = (5 + 2.5) / 2 * 20/3
lam_low = 28/9 / 5
lam_high = 2.5 / 5
l_J_low = 2 / 3 * 5 * (1 + lam_low + lam_low**2) / (1 + lam_low)
l_J_high = 2 / 3 * 5 * (1 + lam_high + lam_high**2) / (1 + lam_high)

Swet_J = 130.78
Swet_J_low = Swet_I * Sref_low / (Sref_low + Sref_high)
Swet_J_high = Swet_I * Sref_high / (Sref_high + Sref_high)

Q_J = 1.07
f_J = 0.12  
maxcamb_loc_J = 0.2

lambda_J_low = np.arctan(2.67/1.17)
lambda_J_high = np.arctan(6.67/3.38)

FF_J_low = getFF(maxcamb_loc_J, f_J , lambda_J_low, 1)
FF_J_high = getFF(maxcamb_loc_J, f_J , lambda_J_high, 1)

Cf_J_low = getCf(l_J_low, 2)
Cf_J_high = getCf(l_J_high, 2)

Cd0_J = getCompCD0(Cf_J_low, FF_J_low, Q_J, Swet_J_low) + getCompCD0(Cf_J_high, FF_J_high, Q_J, Swet_J_high)


# Miscellaneous Form Drag

#No speed breaks or bluff surfaces
#No Fuselage upsweep
#Not a jet

Dq_feathered_electric = 0.1 * 0.3393 * np.pi / 4 * (10/3)**2
Dq_feathered_gas = 0.1 * 0.3393 * np.pi / 4 * (9)**2
Dq_stopped_electric = 0.8 * 0.3393 * np.pi / 4 * (10/3)**2
Dq_stopped_gas = 0.8 * 0.3393 * np.pi / 4 * (9)**2

Cd0_fe = Dq_feathered_electric / S_ref
Cd0_fg = Dq_feathered_gas / S_ref
Cd0_se = Dq_stopped_electric / S_ref
Cd0_sg = Dq_stopped_gas / S_ref


# Sum all Cd0 contributions (including all 5 nacelles)
Total_Cd0 = (
    Cd0_A + Cd0_B + Cd0_C + Cd0_D + Cd0_E + 
    Cd0_F + Cd0_G + np.sum(Cd0_H) + Cd0_I + Cd0_J
)

#Leakage / Protuberance Drag

Cd0_LP = 0.05 * Total_Cd0
Total_Cd0 = Total_Cd0 + Cd0_LP



In [7]:

from tabulate import tabulate  # For better table formatting

# Create a table of component drag contributions
components = [
    ["A: Fuselage", Cd0_A],
    ["B: Main Wing", Cd0_B],
    ["C: Canard", Cd0_C],
    ["D: Canopy", Cd0_D],
    ["E: Booms", Cd0_E],
    ["F: Main Winglet", Cd0_F],
    ["G: Canard Winglet", Cd0_G],
    *[["H: Nacelle "+str(i+1), Cd0_H[i]] for i in range(5)],  # All 5 nacelles
    ["I: Horizontal Stabilizer", Cd0_I],
    ["J: Vertical Stabilizers", Cd0_J],
    ["Leakage and Protuberance Drag", Cd0_LP],
    ["TOTAL Cd0", Total_Cd0]
]

# Print component drag table
print("="*70)
print("AERODYNAMIC DRAG BREAKDOWN")
print("="*70)
print(tabulate(components, headers=["Component", "Cd0"], floatfmt=".6f", tablefmt="grid"))
print("="*70)

# Print engine failure scenarios
failure_scenarios = [
    ["1 Electric engine feathered", Cd0_fe],
    ["1 Gas engine feathered", Cd0_fg],
    ["1 Electric engine stopped", Cd0_se],
    ["1 Gas engine stopped", Cd0_sg]
]

print("\n\nZERO-LIFT DRAG INCREASE FOR ENGINE FAILURE SCENARIOS")
print("="*70)
print(tabulate(failure_scenarios, headers=["Scenario", "ΔCd"], floatfmt=".6f", tablefmt="grid"))
print("="*70)

AERODYNAMIC DRAG BREAKDOWN
+-------------------------------+----------+
| Component                     |      Cd0 |
+===============================+==========+
| A: Fuselage                   | 0.003359 |
+-------------------------------+----------+
| B: Main Wing                  | 0.007732 |
+-------------------------------+----------+
| C: Canard                     | 0.001013 |
+-------------------------------+----------+
| D: Canopy                     | 0.003359 |
+-------------------------------+----------+
| E: Booms                      | 0.000430 |
+-------------------------------+----------+
| F: Main Winglet               | 0.000469 |
+-------------------------------+----------+
| G: Canard Winglet             | 0.000116 |
+-------------------------------+----------+
| H: Nacelle 1                  | 0.000070 |
+-------------------------------+----------+
| H: Nacelle 2                  | 0.000071 |
+-------------------------------+----------+
| H: Nacelle 3              